In [1]:
# Import libraries
import pandas as pd
import numpy as np
import folium
from folium import plugins
import json
from geopy.geocoders import Nominatim
import requests
from pandas import json_normalize
import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.style.use('ggplot')

%matplotlib inline 

print('Libraries imported.')

Libraries imported.


In [2]:
# Read in the data
df = pd.read_csv('data/kc_house_data.csv')
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [3]:
# Get just the first 750
df.drop(df.index[751:], inplace=True)

# Convert id to str
df.id = df.id.astype(str)

# Convert price to int
df.price = df.price.astype(int)

# Create new dataframe
df_sm = df

# Set columns to the correct data types for Folium
df_map = df_sm.astype(str)
df_map.lat = df_map.lat.astype(float)
df_map.long = df_map.long.astype(float)

# Just a heads up
print("Ready to create map")

Ready to create map


In [4]:
df_map.columns

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15'],
      dtype='object')

In [5]:
# Create the map
kc_map = folium.Map(location=[47.49082,-121.835846], zoom_start=8)

# instantiate a mark cluster object for each sale in the dataframe
sales = plugins.MarkerCluster().add_to(kc_map)

# loop through the dataframe and add each data point to the mark cluster
for lt,ln,nm,sqft,sqft_lot,beds,bath,built,price in zip(df_map.lat,
                                                      df_map.long,
                                                      df_map.id,
                                                      df_map.sqft_living, 
                                                      df_map.sqft_lot, 
                                                      df_map.bedrooms, 
                                                      df_map.bathrooms, 
                                                      df_map.yr_built, 
                                                      df_map.price):
    popup = ("<div style=""width:250px;"">Property ID: {nm}<br>" 
        "This is a {sqft} sq ft home on a {sqft_lot} sq ft lot. It has {beds} bedroom, {bath} bathrooms, and was built in {built}. It sold for ${price}.<br><br>"
        "<b>Overview</b><br>" 
        "Beds: {beds}<br>" 
        "Baths: {bath}<br>" 
        "Sq ft: {sqft}<br>" 
        "Lot size: {sqft_lot}: <br>" 
        "Built: {built}<br>" 
        "Price: ${price}<br></div>"
    ).format(nm = nm, 
             sqft = sqft, 
             sqft_lot = sqft_lot, 
             beds = beds, 
             bath = bath, 
             built = built,
             price = price)
    folium.Marker(location=[lt,ln], popup=popup).add_to(sales)

kc_map

In [6]:
#saving map to a html file
kc_map.save('docs/kc_house_sales.html')
print("Map saved!")

Map saved!
